In [75]:
# imports and globals
import numpy as np
from cnns.core import LasagneGoogLeNetInceptionV3 as inceptv3
from cnns.utils import cnn_utils as cu
from cnns.utils import process_utils as pu
from cnns.utils import training_utils as tu
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.updates import nesterov_momentum, adam
from lasagne.layers import get_all_params
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import TrainSplit
from nolearn.lasagne import objective
import matplotlib.pyplot as plt
import pickle as pkl
import os
import csv
import datetime
import sys
import pandas as pd
import json


cnn_dir = '/Users/babasarala/repos/cnns'
eval_dirpath = '/Users/babasarala/Desktop/evaluation'
model_dir = '%s/models'%cnn_dir

%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

Collect data

In [76]:
dataset_dirpath = '%s/dataset/productivity_food_dataset_v1.2'%(eval_dirpath)
csv_filepath = '%s/productivity_food_dataset_v1.2.csv'%(dataset_dirpath)
results_dirpath = '%s/GoogLeNetv3_pool3_fvs'%(dataset_dirpath)
complete_data, feat_cols = construct_complete_data_from_intermediate_results(results_dirpath, csv_filepath)

# set seeds
train_random_seed = np.random.randint(low=0, high=99999)
prod_random_seed = np.random.randint(low=0, high=99999)

# create mappings
categories = ['food', 'documents', 'whiteboards', 'sketches', 'other']
mappings = {}
for idx, category in enumerate(categories):
    mappings[category] = idx
    mappings[idx] = category

# tag training and testing labels
m = complete_data.shape[0]
X = complete_data[feat_cols].values.astype(np.float32)

# simple normalization
X -= X.mean()
X /= X.std()

y = np.array([mappings[category] for category in complete_data['category'].values]).astype(np.int32)

In [94]:
# hyperparameters
input_size = 2048
num_hidden_units = 200
output_size = 5
update_learning_rate = 0.01
update_momentum=0.9
max_epochs=500
optimizer=nesterov_momentum
split_size = 0.25

In [95]:
nnet = NeuralNet(
        layers=[
            # Input layer
            (InputLayer, {'shape': (None, input_size)}),
            (DropoutLayer, {'p':0.2}),
            # Hidden layer
            (DenseLayer, {'num_units': num_hidden_units,
                          'nonlinearity': rectify}),
            (DropoutLayer, {'p':0.5}),    
            # Output (Softmax) layer
            (DenseLayer, {'num_units': output_size, 'nonlinearity': softmax})
        ],

        max_epochs=max_epochs,

        # optimization method
        update=optimizer,
        update_learning_rate=update_learning_rate,
        update_momentum=update_momentum,

        train_split=TrainSplit(eval_size=split_size),
        verbose=1
    )

In [96]:
nnet.fit(X, y)

# Neural Network with 410805 learnable parameters

## Layer information

  #  name        size
---  --------  ------
  0  input0      2048
  1  dropout1    2048
  2  dense2       200
  3  dropout3     200
  4  dense4         5

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       0.33514       8.14629      0.04114      0.29470  1.94s
      2       0.36369      10.26825      0.03542      0.29470  1.93s
      3       0.36558      10.31424      0.03544      0.29470  1.93s
      4       0.38864      10.97472      0.03541      0.29470  3.32s
      5       0.39367      11.24239      0.03502      0.29470  4.44s
      6       0.40660       9.84006      0.04132      0.29470  5.01s
      7       0.39928       9.40580      0.04245      0.29470  5.24s
      8       0.43553       5.51481      0.07898      0.29470  5.56s
      9       0.40383       4.78461      0.08440      0.29470  5.75s
     10       0

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x11024e490>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x11024e350>,
     check_input=True, custom_scores=None,
     layers=[(<class 'lasagne.layers.input.InputLayer'>, {'shape': (None, 2048)}), (<class 'lasagne.layers.noise.DropoutLayer'>, {'p': 0.2}), (<class 'lasagne.layers.dense.DenseLayer'>, {'num_units': 200, 'nonlinearity': <function rectify at 0x10c95ff50>}), (<class 'lasagne.layers.noise.DropoutLayer'>, {'p': 0.5}), (<class 'lasagne.layers.dense.DenseLayer'>, {'num_units': 5, 'nonlinearity': <function softmax at 0x10c95fb90>})],
     loss=None, max_epochs=500, more_params={},
     objective=<function objective at 0x11024bc80>,
     objective_loss_function=<function categorical_crossentropy at 0x10cb0f2a8>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x114a5da28>],
     on_training_finished=[],


In [ ]:
from nolearn.lasagne.visualize import draw_to_notebook
from nolearn.lasagne.visualize import plot_loss
from nolearn.lasagne.visualize import plot_conv_weights
from nolearn.lasagne.visualize import plot_conv_activity
from nolearn.lasagne.visualize import plot_occlusion
from nolearn.lasagne.visualize import plot_saliency

In [ ]:
class LasagneNeuralNetwork():
    def __init__(self, ):
        self